In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
import time
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = 48

In [ ]:
X = pickle.load(open("X-{}.pickle".format(IMAGE_SIZE), "rb"))
y = np.array(pickle.load(open("y-{}.pickle".format(IMAGE_SIZE), "rb")))
X = X/255.0

In [ ]:
X.shape

In [ ]:
dense_layers = [1]
conv_layers =  [3]
layer_sizes =  [128]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "emo-analysis-Adam-decay-lr-10-epoch-{}-conv-{}-nodes-{}-dense-{}.model".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs\{}'.format(NAME))

            model = Sequential() # 128x256 | pooling2x2 | 512 | pooling2x2 | 512 | polling2x2 | dense 1024 | dense 7
 
            model.add(Conv2D(layer_size, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
            model.add(Conv2D(layer_size*2, kernel_size=(3, 3), activation='relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size*4, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(0.25))
            
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(layer_size*4, activation='relu'))
                model.add(Dropout(0.5))  
                
            model.add(Dense(1024, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(7, activation='softmax'))

            model.compile(loss="sparse_categorical_crossentropy", # 7 categories to predict
                                     optimizer=Adam(lr=0.0001, decay=1e-6),  
                                     metrics=['accuracy'])

            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

            model.save("models/"+NAME)

# model with dropout

In [ ]:
from tensorflow.keras.optimizers import Adam

NAME = "emo-analysis-{}-conv-{}-nodes-{}-dense-{}.model".format("Adam-decay-lr-github-4", "34-64-128-128","1024-7", int(time.time()))
tensorboard = TensorBoard(log_dir='logs\{}'.format(NAME))
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", # 7 categories to predict
                         optimizer=Adam(lr=0.0001, decay=1e-6),  
                         metrics=['accuracy'])

model.fit(X, y, batch_size=64, epochs=50, validation_split=0.1, callbacks=[tensorboard])

model.save("models/"+NAME)

In [ ]:
for i in range(1):
    print(i)